<a href="https://colab.research.google.com/github/SelinaZ07/SelinaZ07/blob/main/Canva_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pillow pandas

In [2]:
import pandas as pd

df = pd.read_csv("certificates.csv")

df["Full Name"] = df["First Name"] + " " + df["Surname"]

df.head()


,First Name,Surname,Certificate ID,Certificate URL,Full Name
0,Maria,Delgado,00000AAA,https://certificates.thinkingbeyond.education/...,Maria Delgado
1,Albert,Einstein,00990AAB,https://certificates.thinkingbeyond.education/...,Albert Einstein
2,Emmy,Noether,00030ZAC,https://certificates.thinkingbeyond.education/...,Emmy Noether


In [19]:
from PIL import Image, ImageDraw, ImageFont
import os

canva_color = (0, 191, 99)
text_color = (255, 255, 255)

# Load the certificate template
template_path = "The Template.png"
output_folder = "Canva Project"
os.makedirs(output_folder, exist_ok=True)

# Load fonts (upload these files first)
name_font_path = "GreatVibes-Regular.ttf"
cert_id_font_path = "HammersmithOne-Regular.ttf"
cert_url_font_path ="LibreBaskerville-Regular.ttf"

name_font_size = 140
cert_id_font_size = 35
cert_url_font_size = 20

name_font = ImageFont.truetype(name_font_path, name_font_size)
cert_id_font = ImageFont.truetype(cert_id_font_path, cert_id_font_size)
cert_url_font = ImageFont.truetype(cert_url_font_path, cert_url_font_size)


for index, row in df.iterrows():
    full_name = row["Full Name"]
    certificate_id = row["Certificate ID"]
    certificate_url = row["Certificate URL"]

    img = Image.open(template_path)
    draw = ImageDraw.Draw(img)


    name_bbox = draw.textbbox((0, 0), full_name, font=name_font)
    text_width = name_bbox[2] - name_bbox[0]
    text_height = name_bbox[3] - name_bbox[1]
    image_width, image_height = img.size
    x_position = (image_width - text_width) / 2 # Center horizontally
    y_position = image_height / 2.1  # Adjust as needed
    draw.text((x_position, y_position), full_name, font=name_font, fill=canva_color)

    cert_text = f"Certificate ID: {certificate_id}"
    cert_bbox = draw.textbbox((0, 0), cert_text, font=cert_id_font)
    cert_width = cert_bbox[2] - cert_bbox[0]
    cert_height = cert_bbox[3] - cert_bbox[1]
    cert_x = (image_width - cert_width) / 2
    cert_y = y_position + text_height + 60  # Adjust spacing
    draw.text((cert_x, cert_y), cert_text, font=cert_id_font, fill=canva_color)

    url_bbox = draw.textbbox((0, 0), certificate_url, font=cert_id_font)
    url_width = url_bbox[2] - url_bbox[0]
    url_height = url_bbox[3] - url_bbox[1]
    url_x = (image_width - url_width) / 2 + 380
    url_y = image_height - url_height - 9.2
    draw.text((url_x, url_y), certificate_url, font=cert_url_font, fill=text_color)

    output_path = os.path.join(output_folder, f"{full_name}.png")
    img.save(output_path)



In [8]:
import os
print(os.listdir())

['.config', '.ipynb_checkpoints', 'HammersmithOne-Regular.ttf', 'drive', 'The Template.png', 'GreatVibes-Regular.ttf', 'certificates.csv', 'Canva Project', 'sample_data']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')